In [ ]:
# Setup: install Qiskit (runs automatically in Colab, no-op in Binder)
!pip install -q qiskit qiskit-aer qiskit-ibm-runtime pylatexenc

# CHSH inequality

*Tinatayang paggamit: Dalawang minuto sa isang Heron r2 processor (PAALALA: Ito ay tantya lamang. Maaaring mag-iba ang iyong runtime.)*
## Pinagmulan
Sa tutorial na ito, magsasagawa kayo ng eksperimento sa isang quantum computer upang ipakita ang paglabag sa CHSH inequality gamit ang Estimator primitive.

Ang CHSH inequality, na pinangalanan mula sa mga may-akda na sina Clauser, Horne, Shimony, at Holt, ay ginagamit upang eksperimentong patunayan ang teorama ni Bell (1969). Ang teoremang ito ay nagsasabing ang mga lokal na teoryang may nakatagong variable ay hindi makapaliwanag ng ilang kahihinatnan ng entanglement sa quantum mechanics. Ang paglabag sa CHSH inequality ay ginagamit upang ipakita na ang quantum mechanics ay hindi tugma sa mga lokal na teoryang may nakatagong variable. Ito ay isang mahalagang eksperimento para sa pag-unawa sa pundasyon ng quantum mechanics.

Ang 2022 Nobel Prize for Physics ay iginawad kina Alain Aspect, John Clauser at Anton Zeilinger dahil sa kanilang pioneering work sa quantum information science, at lalo na, para sa kanilang mga eksperimento na may entangled photons na nagpapakita ng paglabag sa mga inequality ni Bell.
## Mga Kinakailangan
Bago magsimula ng tutorial na ito, tiyaking mayroon kayong sumusunod na naka-install:

* Qiskit SDK v1.0 o mas bago, na may [visualization](https://docs.quantum.ibm.com/api/qiskit/visualization) support
* Qiskit Runtime (`pip install qiskit-ibm-runtime`) v0.22 o mas bago
## Setup

In [1]:
# General
import numpy as np

# Qiskit imports
from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
from qiskit.quantum_info import SparsePauliOp
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

# Qiskit Runtime imports
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import EstimatorV2 as Estimator

# Plotting routines
import matplotlib.pyplot as plt
import matplotlib.ticker as tck

## Hakbang 1: I-map ang mga classical input sa isang quantum problem
Para sa eksperimentong ito, gagawa tayo ng isang entangled pair kung saan susukatin natin ang bawat qubit sa dalawang magkaibang bases. Tatakan natin ang mga bases para sa unang qubit na $A$ at $a$ at ang mga bases para sa pangalawang qubit na $B$ at $b$. Ito ay nagbibigay-daan sa atin na kalkulahin ang CHSH quantity na $S_1$:

$$
S_1 = A(B-b) + a(B+b).
$$

Ang bawat observable ay $+1$ o $-1$. Malinaw, ang isa sa mga termino na $B\pm b$ ay dapat na $0$, at ang isa pa ay dapat na $\pm 2$. Samakatuwid, $S_1 = \pm 2$. Ang average value ng $S_1$ ay dapat sumunod sa inequality:

$$
|\langle S_1 \rangle|\leq 2.
$$

Ang pag-expand ng $S_1$ sa mga termino ng $A$, $a$, $B$, at $b$ ay nagreresulta sa:

$$
|\langle S_1 \rangle| = |\langle AB \rangle - \langle Ab \rangle + \langle aB \rangle + \langle ab \rangle| \leq 2
$$

Maaari kayong magtalaga ng isa pang CHSH quantity na $S_2$:

$$
S_2 = A(B+b) - a(B-b),
$$

Ito ay humahantong sa isa pang inequality:

$$
|\langle S_2 \rangle| = |\langle AB \rangle + \langle Ab \rangle - \langle aB \rangle + \langle ab \rangle| \leq 2
$$

Kung ang quantum mechanics ay maaaring ilarawan ng mga lokal na teoryang may nakatagong variable, ang mga nakaraang inequality ay dapat na totoo. Gayunpaman, gaya ng ipinakita sa tutorial na ito, ang mga inequality na ito ay maaaring labagin sa isang quantum computer. Samakatuwid, ang quantum mechanics ay hindi tugma sa mga lokal na teoryang may nakatagong variable.
Kung nais ninyong matuto ng higit pang teorya, tuklasin ang [Entanglement in Action](/learning/courses/basics-of-quantum-information/entanglement-in-action/chsh-game) kasama si John Watrous.
Gagawa kayo ng isang entangled pair sa pagitan ng dalawang qubit sa isang quantum computer sa pamamagitan ng paglikha ng Bell state na $|\Phi^+\rangle = \frac{|00\rangle + |11\rangle}{\sqrt{2}}$. Gamit ang Estimator primitive, maaari ninyong direktang makuha ang mga expectation value na kailangan ($\langle AB \rangle, \langle Ab \rangle, \langle aB \rangle$, at $\langle ab \rangle$) upang kalkulahin ang mga expectation value ng dalawang CHSH quantities na $\langle S_1\rangle$ at $\langle S_2\rangle$. Bago ang pagpapakilala ng Estimator primitive, kailangan ninyong bumuo ng mga expectation value mula sa mga measurement outcomes.

Susukat ninyo ang pangalawang qubit sa $Z$ at $X$ bases. Ang unang qubit ay susukatin din sa orthogonal bases, ngunit may anggulo na may kaugnayan sa pangalawang qubit, na ating i-sweep sa pagitan ng $0$ at $2\pi$. Gaya ng inyong makikita, ang Estimator primitive ay ginagawang napakadaling patakbuhin ang mga parameterized circuits. Sa halip na gumawa ng serye ng mga CHSH circuits, kailangan lamang ninyong gumawa ng *isang* CHSH circuit na may parameter na tumutukoy sa measurement angle at isang serye ng mga phase values para sa parameter.

Sa wakas, susuriin ninyo ang mga resulta at i-plot ang mga ito laban sa measurement angle. Makikita ninyo na para sa ilang hanay ng mga measurement angles, ang mga expectation values ng mga CHSH quantities na $|\langle S_1\rangle| > 2$ o $|\langle S_2\rangle| > 2$, na nagpapakita ng paglabag sa CHSH inequality.

In [2]:
# To run on hardware, select the backend with the fewest number of jobs in the queue
service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=127
)
backend.name

'ibm_kingston'

### Create a parameterized CHSH circuit

First, we write the circuit with the parameter $\theta$, which we call `theta`. The [`Estimator` primitive](https://docs.quantum-computing.ibm.com/api/qiskit-ibm-runtime/qiskit_ibm_runtime.EstimatorV2) can enormously simplify circuit building and output analysis by directly providing expectation values of observables. Many problems of interest, especially for near-term applications on noisy systems, can be formulated in terms of expectation values. `Estimator` (V2) primitive can automatically change measurement basis based on the supplied observable.

In [3]:
theta = Parameter("$\\theta$")

chsh_circuit = QuantumCircuit(2)
chsh_circuit.h(0)
chsh_circuit.cx(0, 1)
chsh_circuit.ry(theta, 0)
chsh_circuit.draw(output="mpl", idle_wires=False, style="iqp")

<Image src="../docs/images/tutorials/chsh-inequality/extracted-outputs/6c77e40a-0.avif" alt="Output of the previous code cell" />

### Gumawa ng isang parameterized CHSH circuit
Una, sumulat tayo ng circuit na may parameter na $\theta$, na ating tatawagin na `theta`. Ang [`Estimator` primitive](https://docs.quantum-computing.ibm.com/api/qiskit-ibm-runtime/qiskit_ibm_runtime.EstimatorV2) ay lubhang makakapag-simplify ng circuit building at output analysis sa pamamagitan ng direktang pagbibigay ng mga expectation values ng mga observables. Maraming problema na nakakainteres, lalo na para sa near-term applications sa maingay na mga sistema, ay maaaring i-formulate sa mga termino ng mga expectation values. Ang `Estimator` (V2) primitive ay awtomatikong makakapagbago ng measurement basis batay sa ibinigay na observable.

In [4]:
number_of_phases = 21
phases = np.linspace(0, 2 * np.pi, number_of_phases)
# Phases need to be expressed as list of lists in order to work
individual_phases = [[ph] for ph in phases]

![Output of the previous code cell](../docs/images/tutorials/chsh-inequality/extracted-outputs/6c77e40a-0.avif)

### Gumawa ng listahan ng mga phase values na italaga sa ibang pagkakataon
Pagkatapos gumawa ng parameterized CHSH circuit, gagawa kayo ng listahan ng mga phase values na italaga sa circuit sa susunod na hakbang. Maaari ninyong gamitin ang sumusunod na code upang gumawa ng listahan ng 21 phase values mula $0$ hanggang $2 \pi$ na may pantay na espasyo, iyon ay, $0$, $0.1 \pi$, $0.2 \pi$, ..., $1.9 \pi$, $2 \pi$.

In [5]:
# <CHSH1> = <AB> - <Ab> + <aB> + <ab> -> <ZZ> - <ZX> + <XZ> + <XX>
observable1 = SparsePauliOp.from_list(
    [("ZZ", 1), ("ZX", -1), ("XZ", 1), ("XX", 1)]
)

# <CHSH2> = <AB> + <Ab> - <aB> + <ab> -> <ZZ> + <ZX> - <XZ> + <XX>
observable2 = SparsePauliOp.from_list(
    [("ZZ", 1), ("ZX", 1), ("XZ", -1), ("XX", 1)]
)

### Observables
Ngayon kailangan natin ng mga observables kung saan kakalkulahin ang mga expectation values. Sa ating kaso, tinitingnan natin ang orthogonal bases para sa bawat qubit, na hinahayaan ang parameterized na $Y-$ rotation para sa unang qubit na i-sweep ang measurement basis na halos patuloy na may kaugnayan sa pangalawang qubit basis. Samakatuwid, pipiliin natin ang mga observables na $ZZ$, $ZX$, $XZ$, at $XX$.

In [6]:
target = backend.target
pm = generate_preset_pass_manager(target=target, optimization_level=3)

chsh_isa_circuit = pm.run(chsh_circuit)
chsh_isa_circuit.draw(output="mpl", idle_wires=False, style="iqp")

<Image src="../docs/images/tutorials/chsh-inequality/extracted-outputs/9a5561eb-0.avif" alt="Output of the previous code cell" />

## Hakbang 2: I-optimize ang problema para sa quantum hardware execution

Upang bawasan ang kabuuang oras ng job execution, ang mga V2 primitives ay tumatanggap lamang ng mga circuits at observables na sumusunod sa mga instruction at connectivity na sinusuportahan ng target system (tinutukoy bilang instruction set architecture (ISA) circuits at observables).

### ISA Circuit

In [7]:
isa_observable1 = observable1.apply_layout(layout=chsh_isa_circuit.layout)
isa_observable2 = observable2.apply_layout(layout=chsh_isa_circuit.layout)

![Output of the previous code cell](../docs/images/tutorials/chsh-inequality/extracted-outputs/9a5561eb-0.avif)

### ISA Observables

Katulad nito, kailangan nating i-transform ang mga observables upang gawing compatible sa backend bago magpatakbo ng mga jobs gamit ang [`Runtime Estimator V2`](https://docs.quantum.ibm.com/api/qiskit-ibm-runtime/estimator-v2#run). Maaari nating isagawa ang transformation gamit ang `apply_layout` method ng `SparsePauliOp` object.

In [8]:
# To run on a local simulator:
# Use the StatevectorEstimator from qiskit.primitives instead.

estimator = Estimator(mode=backend)

pub = (
    chsh_isa_circuit,  # ISA circuit
    [[isa_observable1], [isa_observable2]],  # ISA Observables
    individual_phases,  # Parameter values
)

job_result = estimator.run(pubs=[pub]).result()

## Hakbang 3: Isagawa gamit ang mga Qiskit primitives
Upang maisagawa ang buong eksperimento sa isang tawag sa [`Estimator`](https://docs.quantum-computing.ibm.com/api/qiskit-ibm-runtime/qiskit_ibm_runtime.EstimatorV2).
Maaari tayong gumawa ng [Qiskit Runtime `Estimator`](https://docs.quantum.ibm.com/api/qiskit-ibm-runtime/estimator-v2) primitive upang kalkulahin ang ating mga expectation values. Ang `EstimatorV2.run()` method ay tumatanggap ng isang iterable ng `primitive unified blocs (PUBs)`. Ang bawat PUB ay isang iterable sa format na `(circuit, observables, parameter_values: Optional, precision: Optional)`.

In [9]:
chsh1_est = job_result[0].data.evs[0]
chsh2_est = job_result[0].data.evs[1]

In [10]:
fig, ax = plt.subplots(figsize=(10, 6))

# results from hardware
ax.plot(phases / np.pi, chsh1_est, "o-", label="CHSH1", zorder=3)
ax.plot(phases / np.pi, chsh2_est, "o-", label="CHSH2", zorder=3)

# classical bound +-2
ax.axhline(y=2, color="0.9", linestyle="--")
ax.axhline(y=-2, color="0.9", linestyle="--")

# quantum bound, +-2√2
ax.axhline(y=np.sqrt(2) * 2, color="0.9", linestyle="-.")
ax.axhline(y=-np.sqrt(2) * 2, color="0.9", linestyle="-.")
ax.fill_between(phases / np.pi, 2, 2 * np.sqrt(2), color="0.6", alpha=0.7)
ax.fill_between(phases / np.pi, -2, -2 * np.sqrt(2), color="0.6", alpha=0.7)

# set x tick labels to the unit of pi
ax.xaxis.set_major_formatter(tck.FormatStrFormatter("%g $\\pi$"))
ax.xaxis.set_major_locator(tck.MultipleLocator(base=0.5))

# set labels, and legend
plt.xlabel("Theta")
plt.ylabel("CHSH witness")
plt.legend()
plt.show()

<Image src="../docs/images/tutorials/chsh-inequality/extracted-outputs/f6267448-0.avif" alt="Output of the previous code cell" />

In the figure, the lines and gray areas delimit the bounds; the outer-most (dash-dotted) lines delimit the quantum-bounds ($\pm 2$), whereas the inner (dashed) lines delimit the classical bounds ($\pm 2\sqrt{2}$). You can see that there are regions where the CHSH witness quantities exceeds the classical bounds. Congratulations! You have successfully demonstrated the violation of CHSH inequality in a real quantum system!

## Tutorial survey

Please take this short survey to provide feedback on this tutorial. Your insights will help us improve our content offerings and user experience.

[Link to survey](https://your.feedback.ibm.com/jfe/form/SV_3xxAgm1SF1wGp9k)